In [0]:
pip install openai pandas hdbscan plotly python-dotenv umap-learn langchain langchain_community -q

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
pip install -U scikit-learn -q

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
import hdbscan
import numpy as np
import pandas as pd
import plotly.express as px

from umap import UMAP
from ast import literal_eval

from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, DoubleType

from sklearn.cluster import DBSCAN
import hdbscan
import umap
import matplotlib.pyplot as plt


from sklearn.metrics import make_scorer
from hdbscan.validity import validity_index

from sklearn.metrics import silhouette_score

In [0]:
%run /Workspace/Users/arestrepo15@summa-sci.com/proyecto_integrador/connection_datalake

[FileInfo(path='abfss://presentation@sadatalakeproyecto.dfs.core.windows.net/dataset_videos_youtube_embeddings.csv', name='dataset_videos_youtube_embeddings.csv', size=689874729, modificationTime=1715291158000),
 FileInfo(path='abfss://presentation@sadatalakeproyecto.dfs.core.windows.net/dataset_videos_youtube_embeddings_300_tokens.csv', name='dataset_videos_youtube_embeddings_300_tokens.csv', size=1924220793, modificationTime=1717730763000),
 FileInfo(path='abfss://presentation@sadatalakeproyecto.dfs.core.windows.net/dataset_videos_youtube_embeddings_500_tokens.csv', name='dataset_videos_youtube_embeddings_500_tokens.csv', size=1292213144, modificationTime=1717730158000),
 FileInfo(path='abfss://presentation@sadatalakeproyecto.dfs.core.windows.net/df_curated_videos_youtube.csv', name='df_curated_videos_youtube.csv', size=91124952, modificationTime=1717729824000),
 FileInfo(path='abfss://presentation@sadatalakeproyecto.dfs.core.windows.net/df_split_250_embeddings_videos_youtube.csv', n

In [0]:
path = "df_split_250_embeddings_videos_youtube.csv"
file_path = f"abfss://presentation@sadatalakeproyecto.dfs.core.windows.net/{path}"
df_youtube_data = spark.read.option("header", True) \
                           .option("inferSchema", True) \
                           .option("sep", ";") \
                           .csv(file_path)

In [0]:

from pyspark.sql.types import ArrayType, DoubleType
import ast

# Definir la UDF para convertir la cadena de texto a un array de floats
def convert_to_array(embedding_str):
    return np.array(ast.literal_eval(embedding_str), dtype=float).tolist()

convert_to_array_udf = udf(convert_to_array, ArrayType(DoubleType()))

# Aplicar la UDF a la columna 'embedding'
df_youtube_data = df_youtube_data.withColumn("embedding", convert_to_array_udf(df_youtube_data["embedding"]))

In [0]:
embeddings = np.array(df_youtube_data.select("embedding").rdd.map(lambda row: row[0]).collect())

In [0]:
from umap import UMAP
# Ajuste del modelo UMAP
umap_model = UMAP(n_components=2, random_state=42, n_neighbors=10, min_dist=0.1, metric='cosine')
umap_embeddings = umap_model.fit_transform(embeddings)

/local_disk0/.ephemeral_nfs/envs/pythonEnv-d797bde5-056d-4907-b9ea-6483bb80b120/lib/python3.10/site-packages/umap/umap_.py:1945: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [0]:
# Graficar los resultados utilizando Plotly
fig = px.scatter(
    x=umap_embeddings[:,0],
    y=umap_embeddings[:,1],
    title="UMAP embeddings",
    width=900,
    height=900,
    color_discrete_sequence=px.colors.qualitative.G10_r
)
fig.show()

In [0]:

# Definir los valores de los hiperparámetros para HDBSCAN
param_grid_hdbscan = {
    'min_cluster_size': [5, 10, 50, 100, 200],
    'min_samples': [5, 100, 300, 600],
    'cluster_selection_method': ['eom', 'leaf']
}

# Crear un DataFrame para almacenar los resultados
results_hdbscan_df = pd.DataFrame(columns=[
    'min_cluster_size', 'min_samples', 
    'cluster_selection_method', 'silhouette_score', 'percent_data_retained', 
    'total_clusters', 'cluster_sizes', 'relative_validity'
])

# Iterar sobre los valores de los hiperparámetros
for min_cluster_size in param_grid_hdbscan['min_cluster_size']:
    for min_samples in param_grid_hdbscan['min_samples']:
        for cluster_selection_method in param_grid_hdbscan['cluster_selection_method']:
            # Entrenar el modelo HDBSCAN
            hdbscan_model = hdbscan.HDBSCAN(
                gen_min_span_tree=True,
                min_cluster_size=min_cluster_size,
                min_samples=min_samples,
                cluster_selection_method=cluster_selection_method
            )
            hdbscan_labels = hdbscan_model.fit_predict(umap_embeddings)
            
            # Calcular métricas
            labels = hdbscan_labels
            clustered = (labels >= 0)
            if len(set(labels)) > 1:
                silhouette = silhouette_score(umap_embeddings, labels, metric='cosine')
            else:
                silhouette = None
            coverage = np.sum(clustered) / umap_embeddings.shape[0]
            total_clusters = np.max(labels) + 1
            cluster_sizes = np.bincount(labels[clustered]).tolist()
            relative_validity = hdbscan_model.relative_validity_
            
            # Crear un DataFrame temporal para el resultado actual
            temp_df = pd.DataFrame({
                'min_cluster_size': [min_cluster_size],
                'min_samples': [min_samples],
                'cluster_selection_method': [cluster_selection_method],
                'silhouette_score': [silhouette],
                'percent_data_retained': [coverage],
                'total_clusters': [total_clusters],
                'cluster_sizes': [cluster_sizes],
                'relative_validity': [relative_validity]
            })
            print(temp_df)
            
            # Concatenar el DataFrame temporal al DataFrame principal
            results_hdbscan_df = pd.concat([results_hdbscan_df, temp_df], ignore_index=True)


   min_cluster_size  ...  relative_validity
0                 5  ...            0.18041

[1 rows x 8 columns]
   min_cluster_size  ...  relative_validity
0                 5  ...           0.170997

[1 rows x 8 columns]
   min_cluster_size  min_samples  ...   cluster_sizes  relative_validity
0                 5          100  ...  [16258, 42747]           0.384407

[1 rows x 8 columns]
   min_cluster_size  ...  relative_validity
0                 5  ...           0.112364

[1 rows x 8 columns]
   min_cluster_size  min_samples  ...   cluster_sizes  relative_validity
0                 5          300  ...  [42738, 16267]           0.275529

[1 rows x 8 columns]
   min_cluster_size  ...  relative_validity
0                 5  ...            0.10559

[1 rows x 8 columns]
   min_cluster_size  min_samples  ...   cluster_sizes  relative_validity
0                 5          600  ...  [16273, 42732]           0.191362

[1 rows x 8 columns]
   min_cluster_size  ...  relative_validity
0           

In [0]:
results_hdbscan_df

,min_cluster_size,min_samples,cluster_selection_method,silhouette_score,percent_data_retained,total_clusters,cluster_sizes,relative_validity
0,5,5,eom,-0.822265,0.564048,2083,"[10, 15, 43, 9, 9, 7, 11, 9, 74, 19, 9, 12, 30...",0.180410
1,5,5,leaf,-0.825244,0.471637,2549,"[10, 15, 9, 9, 7, 11, 9, 19, 9, 5, 24, 12, 30,...",0.170997
2,5,100,eom,-0.179256,0.998849,2,"[16258, 42747]",0.384407
3,5,100,leaf,-0.771145,0.248828,101,"[217, 47, 39, 477, 138, 52, 106, 117, 169, 254...",0.112364
4,5,300,eom,-0.178968,0.998849,2,"[42738, 16267]",0.275529
5,5,300,leaf,-0.705273,0.238129,39,"[1168, 6, 174, 77, 1180, 616, 56, 246, 388, 48...",0.105590
6,5,600,eom,-0.178777,0.998849,2,"[16273, 42732]",0.191362
7,5,600,leaf,-0.684012,0.236301,24,"[1017, 44, 1084, 172, 29, 538, 716, 182, 720, ...",0.157996
8,10,5,eom,-0.888538,0.881858,294,"[10, 15, 43, 11, 74, 29, 43, 12, 30, 85, 15, 2...",0.051766
9,10,5,leaf,-0.798217,0.470672,1366,"[10, 15, 43, 11, 29, 43, 12, 30, 15, 20, 17, 1...",0.185995


In [0]:

min_cluster_size=200
min_samples=100
cluster_selection_method="eom"
hdbscan_model = hdbscan.HDBSCAN(
    gen_min_span_tree=True,
    min_cluster_size=min_cluster_size,
    min_samples=min_samples,
    cluster_selection_method=cluster_selection_method
)
hdbscan_labels = hdbscan_model.fit_predict(umap_embeddings)

In [0]:
# Graficar los resultados utilizando Plotly
fig = px.scatter(
    x=umap_embeddings[:,0],
    y=umap_embeddings[:,1],
    color=hdbscan_labels,
    title="UMAP embeddings",
    width=900,
    height=900,
    color_discrete_sequence=px.colors.qualitative.G10_r
)
fig.show()

In [0]:
# Convertir el DataFrame de Spark a un DataFrame de pandas
df_youtube = df_youtube_data.toPandas()

# Agregar la columna de etiquetas al DataFrame de pandas
df_youtube['hdbscan_labels_umap'] = hdbscan_labels

In [0]:
for c in set(hdbscan_labels):
    print(f"**********************************cluster: {c}************************************************")
    print(df_youtube[df_youtube['hdbscan_labels_umap']==c]['text'].head().values)

**********************************cluster: 0************************************************
['bitcoin y sp500 reaccionan mal a refugios largo plazo | repaso a mercado y cartera de acciones. quiero comenzar el repaso de la bolsa hoy sábado y también del premercado que será a partir del martes cuando podamos empezar a negociar la semana y además también el año hablándote de una incorporación a la cartera que haré precisamente a partir del martes es del sector de la biotecnología y es que invertir en biotecnología es un desafío debida a las grandes disparidades en el éxito y el fracaso y los resultados del desarrollo de fármacos que son binarios cuando hablo de resultados binarios me refiero a la regla del 9090 se estima que el 90 por de los medicamentos que entran en ensayos clínicos en humanos no logran la aprobación y los pronósticos de ventas de los medicamentos aprobados son incorrecto el 90 por de las pces pero mira a pesar de eso a pesar de los desafíos que da este sector creo que

In [0]:
df_youtube[df_youtube['hdbscan_labels_umap']==2]['text']

5079     internet ni nada pero te diré cómo puedes part...
11063    dos años y yo opero a través de bitget si uste...
11105    pero muy atentos la idea es jodernos tengamos ...
11106    serio y muy responsable que me apoya y además ...
11128    que han hecho 100000 dólares de volumen para p...
                               ...                        
58913    perdiendo más porque ahí van a empezar a salir...
58914    lenguaje más inclusivo y amigable para las per...
58915    además viene un concurso también a través de b...
58962    referidos el cual permanece aquí en la descrip...
58967    roja que lo va a intentar hacer cambiar de opi...
Name: text, Length: 1186, dtype: object

In [0]:
df_youtube[df_youtube['hdbscan_labels_umap']==1]['text']

8342     está muy próxima que le cierren la cuenta bye bye
11126    aprendo de ustedes y me he fortalecido y he te...
11230     nos vemos okay cuídense cuiden su dinero bye bye
11517    pues solana rebotó sola okay eder bueno mis am...
11652     nocivo para su bolsillo nos vemos mañana bye bye
                               ...                        
58562    a quién que dios los bendiga y nos vemos mañan...
58699    no lo notó eh ezequiel álvarez carlos blanco t...
58735    cuidan que dios los bendiga nos vemos mañana m...
58951    pórtese bien haga el bien y no mire a quién y ...
59072    green qué bien no yo me imagino que ustedes es...
Name: text, Length: 217, dtype: object

In [0]:
df_youtube[['hdbscan_labels_umap']].value_counts()

hdbscan_labels_umap
 4                     39430
 0                     16258
 3                      1251
 2                      1186
-1                       731
 1                       217
dtype: int64